# Title: Spatio-temporal visualization of Asian Carp count in Brandon pool

## Author: Gargi Chaudhuri, Dept. of Geography and Earth Science, University of Wisconsin-La Crosse

### Descrition: This script reads a CSV file with correct latitude/longitude, parses data into weekly aggregates and creates spatio-temporal animation of Asian carp counts in Brandon pool, IL based on Asian Carp catch data by UMESC.

#### Import all the required packages

In [2]:
import os
import pandas as pd
import folium
from folium import plugins
from datetime import datetime

#### Set the working directory:
Currently set to Gargi's working folder. Change accordingly.

In [36]:
os.chdir("C:/Users/gchaudhuri/OneDrive - University of Wisconsin-La Crosse/research_stuff/umesc_fish_work/summer_2019_work")
data="https:/github.com/gchaudhuri/asian_carp_viz/blob/master/brandon_carp_2011_2018_gcs.csv"

## Data clean-up:

#### Read the fish data file:
The file should have correct geospatial location and should be in csv file. The file here contains all data from 2011 to 2018. The columns listed under "usecols=" are the only ones used. 

In [4]:
df = pd.read_csv("brandon_carp_2011_2018_gcs.csv", usecols= ['OBJECTID', 'barcode', 'fish_count','site_alias', 'start_date',
                                                             'start_time', 'finish_date', 'finish_time', 'adj_lat',
                                                             'adj_long2','x', 'y'])

#### Convert data/time column to pandas format:
Convert the time/date columns into Pandas format, so that the code can automatically detect and aggregate data based on week/month/year. 

In [5]:
df.start_date = pd.to_datetime(df.start_date, format='%m/%d/%Y')
df.finish_date = pd.to_datetime(df.finish_date, format='%m/%d/%Y')
df.start_time = pd.to_datetime(df.start_time, format='%H:%M:%S')
df.finish_time = pd.to_datetime(df.finish_time, format='%H:%M:%S')

Now add <code>year</code>, <code>month</code>, <code>week</code>, and <code>day</code>, to assign numbers for each temporal unit.

In [6]:
df.sort_values(by=['start_date', 'start_time'])
df['year'] = df.start_date.apply(lambda x: x.year)
df['month'] = df.start_date.apply(lambda x: x.month)
df['week'] = df.start_date.apply(lambda x: x.week)
df['day'] = df.start_date.apply(lambda x: x.day)

#### Float format for locational data:
Convert the locational data in float format. In the oresent dataset, the GPS locations are in <code>adj_lat</code> and <code>adj_long</code> column. 

In [7]:
df['adj_lat'] = df['adj_lat'].astype(float)
df['adj_long2'] = df['adj_long2'].astype(float)

#### Finally print to check data:
Print to check everything in order. In the original dataset, the <code>start_time</code> and <code>finish_time</code> were incomplete or incorrect and here 1900-XX-XX dates as prefix randomly. Please ignore that or can even delete these columns. 

In [8]:
print(df.head())

   OBJECTID          barcode  fish_count site_alias start_date  \
0        26  040811BRPLtra04           7    BRPLN04 2011-04-08   
1        29  040811BRPLtra03           4    BRPLN03 2011-04-08   
2        33  040811BRPLtra01           1    BRPLN01 2011-04-08   
3        36  040811BRPLtra02         107    BRPLN02 2011-04-08   
4         7  052411BRPLefd01           2    BRPLE01 2011-05-24   

           start_time finish_date finish_time    adj_lat  adj_long2  \
0 1900-01-01 00:00:00  2011-04-08  1900-01-01  41.552550 -88.079201   
1 1900-01-01 05:45:00  2011-04-08  1900-01-01  41.539567 -88.082100   
2 1900-01-01 06:15:00  2011-04-08  1900-01-01  41.517217 -88.088799   
3 1900-01-01 07:30:00  2011-04-08  1900-01-01  41.509267 -88.092201   
4 1900-01-01 16:41:00  2011-05-24  1900-01-01  41.567100 -88.077797   

             x            y  year  month  week  day  
0  910435.0281  4611804.481  2011      4    14    8  
1  910275.4368  4610348.657  2011      4    14    8  
2  909852.1349

#### However, if there is a dataset with correct and complete start and finish time:
If the print start and finish time are correct and complete, then the time taken to collect the data can be used to normalize the data collection rate. To do so, you have to calculate the time taken to collect data during each entry, bu using the following code:
<code>
t2 = df.finish_time
t1 = df.start_time
df['time_dif']= (t2 - t1)
</code>

For this dataset, the columns have been deleted and the new dataframe is named as <code>df_notime</code>. 

In [9]:
df_notime = df.drop(columns=['start_time', 'finish_time'])
print(df_notime.head())
print(df_notime.tail())

   OBJECTID          barcode  fish_count site_alias start_date finish_date  \
0        26  040811BRPLtra04           7    BRPLN04 2011-04-08  2011-04-08   
1        29  040811BRPLtra03           4    BRPLN03 2011-04-08  2011-04-08   
2        33  040811BRPLtra01           1    BRPLN01 2011-04-08  2011-04-08   
3        36  040811BRPLtra02         107    BRPLN02 2011-04-08  2011-04-08   
4         7  052411BRPLefd01           2    BRPLE01 2011-05-24  2011-05-24   

     adj_lat  adj_long2            x            y  year  month  week  day  
0  41.552550 -88.079201  910435.0281  4611804.481  2011      4    14    8  
1  41.539567 -88.082100  910275.4368  4610348.657  2011      4    14    8  
2  41.517217 -88.088799  909852.1349  4607834.059  2011      4    14    8  
3  41.509267 -88.092201  909618.9745  4606934.923  2011      4    14    8  
4  41.567100 -88.077797  910454.7282  4613426.942  2011      5    21   24  
      OBJECTID   barcode  fish_count         site_alias start_date  \
1116 

#### Check data:
Make sure the data types are in right format and the number of data points are correct. 

In [10]:
print(df_notime.dtypes)
print(df_notime.info())

OBJECTID                int64
barcode                object
fish_count              int64
site_alias             object
start_date     datetime64[ns]
finish_date    datetime64[ns]
adj_lat               float64
adj_long2             float64
x                     float64
y                     float64
year                    int64
month                   int64
week                    int64
day                     int64
dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1121 entries, 0 to 1120
Data columns (total 14 columns):
OBJECTID       1121 non-null int64
barcode        1121 non-null object
fish_count     1121 non-null int64
site_alias     1121 non-null object
start_date     1121 non-null datetime64[ns]
finish_date    1121 non-null datetime64[ns]
adj_lat        1121 non-null float64
adj_long2      1121 non-null float64
x              1121 non-null float64
y              1121 non-null float64
year           1121 non-null int64
month          1121 non-null int64
week       

#### Save the changes as a csv file:

In [11]:
df_notime.to_csv("brandon_carp_2011_2018_edited.csv")

## Mapping:

#### Start an empty map with lat/long of Joliet, IL and add a basemap:

In [12]:
map = folium.Map(location=[41.526811, -88.086320], zoom_start = 13) 

def generateBaseMap(default_location=[41.526811, -88.086320], default_zoom_start=13):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

### Plot data:

#### Create a heatmap with timeseries of 2011 by week:

In [30]:
# Create a copy of 2011 data from the original dataset:
df_2011 = df_notime[df_notime.year==2011].copy()

#Initiate an empty list for 2011:
df_2011_week_list = []

# Populate the list with data grouped by weekly totals:
for week in df_2011.week.sort_values().unique():
    df_2011_week_list.append(df_2011.loc[df_2011.week == week, ['adj_lat', 'adj_long2', 'fish_count', 'week', 'year']].groupby(['adj_lat', 'adj_long2']).sum().reset_index().values.tolist())

#Create date labels for the time slider:
index = ['Week and Year '+str(row[1][3:5]) for row in df_2011_week_list]

#Generate an empty basemap:
base_map2011= generateBaseMap(default_zoom_start=13)

#Visualize the 2011 datapoints with heatmap timeseries:
from folium.plugins import HeatMapWithTime
HeatMapWithTime(df_2011_week_list, index, radius=8, gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'}, min_opacity=0.5, max_opacity=0.8, use_local_extrema=True).add_to(base_map2011)

#Display the map for 2011:
base_map2011

#### Create a copy of 2012 data points, sort by week, and create an animation:

In [26]:
# Create a copy of 2012 data from the original dataset:
df_2012 = df_notime[df_notime.year==2012].copy()

#Initiate an empty list for 2012:
df_2012_week_list = []

# Populate the list with data grouped by weekly totals:
for week in df_2012.week.sort_values().unique():
    df_2012_week_list.append(df_2012.loc[df_2012.week == week, ['adj_lat', 'adj_long2', 'fish_count', 'week', 'year']].groupby(['adj_lat', 'adj_long2']).sum().reset_index().values.tolist())

#Create date labels for the time slider:
index = ['Week and Year '+str(row[0][3:5]) for row in df_2012_week_list]

#Generate an empty basemap:
base_map2012= generateBaseMap(default_zoom_start=13)

#Visualize the 2012 datapoints with heatmap timeseries:
from folium.plugins import HeatMapWithTime
HeatMapWithTime(df_2012_week_list, index, radius=8, gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'}, min_opacity=0.5, max_opacity=0.8, use_local_extrema=True).add_to(base_map2012)

#Display the map for 2012:
base_map2012

#### Create a copy of 2013 data points, sort by week, and create an animation:

In [31]:
# Create a copy of 2013 data from the original dataset:
df_2013 = df_notime[df_notime.year==2013].copy()

#Initiate an empty list for 2013:
df_2013_week_list = []

# Populate the list with data grouped by weekly totals:
for week in df_2013.week.sort_values().unique():
    df_2013_week_list.append(df_2013.loc[df_2013.week == week, ['adj_lat', 'adj_long2', 'fish_count', 'week', 'year']].groupby(['adj_lat', 'adj_long2']).sum().reset_index().values.tolist())

#Create date labels for the time slider:
index = ['Week and Year '+str(row[0][3:5]) for row in df_2013_week_list]

#Generate an empty basemap:
base_map2013= generateBaseMap(default_zoom_start=13)

#Visualize the 2013 datapoints with heatmap timeseries:
from folium.plugins import HeatMapWithTime
HeatMapWithTime(df_2013_week_list, index, radius=8, gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'}, min_opacity=0.5, max_opacity=0.8, use_local_extrema=True).add_to(base_map2013)

#Display the map for 2013:
base_map2013

#### Create a copy of 2014 data points, sort by week, and create an animation:

In [32]:
# Create a copy of 2014 data from the original dataset:
df_2014 = df_notime[df_notime.year==2014].copy()

#Initiate an empty list for 2014:
df_2014_week_list = []

# Populate the list with data grouped by weekly totals:
for week in df_2014.week.sort_values().unique():
    df_2014_week_list.append(df_2014.loc[df_2014.week == week, ['adj_lat', 'adj_long2', 'fish_count', 'week', 'year']].groupby(['adj_lat', 'adj_long2']).sum().reset_index().values.tolist())

#Create date labels for the time slider:
index = ['Week and Year '+str(row[0][3:5]) for row in df_2014_week_list]

#Generate an empty basemap:
base_map2014= generateBaseMap(default_zoom_start=13)

#Visualize the 2014 datapoints with heatmap timeseries:
from folium.plugins import HeatMapWithTime
HeatMapWithTime(df_2014_week_list, index, radius=8, gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'}, min_opacity=0.5, max_opacity=0.8, use_local_extrema=True).add_to(base_map2014)

#Display the map for 2014:
base_map2014

#### Create a copy of 2015 data points, sort by week, and create an animation:

In [33]:
# Create a copy of 2015 data from the original dataset:
df_2015 = df_notime[df_notime.year==2015].copy()

#Initiate an empty list for 2015:
df_2015_week_list = []

# Populate the list with data grouped by weekly totals:
for week in df_2015.week.sort_values().unique():
    df_2015_week_list.append(df_2015.loc[df_2015.week == week, ['adj_lat', 'adj_long2', 'fish_count', 'week', 'year']].groupby(['adj_lat', 'adj_long2']).sum().reset_index().values.tolist())

#Create date labels for the time slider:
index = ['Week and Year '+str(row[0][3:5]) for row in df_2015_week_list]

#Generate an empty basemap:
base_map2015= generateBaseMap(default_zoom_start=13)

#Visualize the 2015 datapoints with heatmap timeseries:
from folium.plugins import HeatMapWithTime
HeatMapWithTime(df_2015_week_list, index, radius=8, gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'}, min_opacity=0.5, max_opacity=0.8, use_local_extrema=True).add_to(base_map2015)

#Display the map for 2015:
base_map2015

#### Create a copy of 2016 data points, sort by week, and create an animation:

In [34]:
# Create a copy of 2016 data from the original dataset:
df_2016 = df_notime[df_notime.year==2016].copy()

#Initiate an empty list for 2016:
df_2016_week_list = []

# Populate the list with data grouped by weekly totals:
for week in df_2016.week.sort_values().unique():
    df_2016_week_list.append(df_2016.loc[df_2016.week == week, ['adj_lat', 'adj_long2', 'fish_count', 'week', 'year']].groupby(['adj_lat', 'adj_long2']).sum().reset_index().values.tolist())

#Create date labels for the time slider:
index = ['Week and Year '+str(row[0][3:5]) for row in df_2016_week_list]

#Generate an empty basemap:
base_map2016= generateBaseMap(default_zoom_start=13)

#Visualize the 2016 datapoints with heatmap timeseries:
from folium.plugins import HeatMapWithTime
HeatMapWithTime(df_2016_week_list, index, radius=8, gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'}, min_opacity=0.5, max_opacity=0.8, use_local_extrema=True).add_to(base_map2016)

#Display the map for 2016:
base_map2016

#### Create a copy of 2017 data points, sort by week, and create an animation:

In [35]:
# Create a copy of 2017 data from the original dataset:
df_2017 = df_notime[df_notime.year==2017].copy()

#Initiate an empty list for 2017:
df_2017_week_list = []

# Populate the list with data grouped by weekly totals:
for week in df_2017.week.sort_values().unique():
    df_2017_week_list.append(df_2017.loc[df_2017.week == week, ['adj_lat', 'adj_long2', 'fish_count', 'week', 'year']].groupby(['adj_lat', 'adj_long2']).sum().reset_index().values.tolist())

#Create date labels for the time slider:
index = ['Week and Year '+str(row[0][3:5]) for row in df_2017_week_list]

#Generate an empty basemap:
base_map2017= generateBaseMap(default_zoom_start=13)

#Visualize the 2017 datapoints with heatmap timeseries:
from folium.plugins import HeatMapWithTime
HeatMapWithTime(df_2017_week_list, index, radius=8, gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'}, min_opacity=0.5, max_opacity=0.8, use_local_extrema=True).add_to(base_map2017)

#Display the map for 2017:
base_map2017

#### Create a copy of 2018 data points, sort by week, and create an animation:

In [28]:
# Create a copy of 2018 data points:
df_2018 = df_notime[df_notime.year==2018].copy()

#Sort values by week in the year 2018
df_2018.sort_values(by=['week']);

#Initiate an empty list for 2018:
df_2018_week_list = []

# Populate the list with data grouped by weekly totals:
for week in df_2018.week.sort_values().unique():
    df_2018_week_list.append(df_2018.loc[df_2018.week == week, ['adj_lat', 'adj_long2', 'fish_count', 'week', 'year']].groupby(['adj_lat', 'adj_long2']).sum().reset_index().values.tolist())

#Create date labels for the time slider:
index = ['Week and Year'+str(row[0][3:5]) for row in df_2018_week_list]

#Generate an empty basemap
base_map2018 = generateBaseMap(default_zoom_start=13)

# Visualize the time series:
from folium.plugins import HeatMapWithTime
HeatMapWithTime(df_2018_week_list, index, radius=8, gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'}, min_opacity=0.5, max_opacity=0.8, use_local_extrema=True, display_index=True).add_to(base_map2018)

# Display map
base_map2018

#### Save the map in HTML format:

In [18]:
base_map2011.save('Heat_Map_animation_2011.html');
base_map2012.save('Heat_Map_animation_2012.html');
base_map2013.save('Heat_Map_animation_2013.html');
base_map2014.save('Heat_Map_animation_2014.html');
base_map2015.save('Heat_Map_animation_2015.html');
base_map2016.save('Heat_Map_animation_2016.html');
base_map2017.save('Heat_Map_animation_2017.html');
base_map2018.save('Heat_Map_animation_2018.html');